<img align=right src="images/inmas.png" width=130x />

# Notebook 03a - Multiple Linear Regression

Material covered in this notebook:

This notebook follows along the notes [here](Notes/3_MultipleLinearRegression.pdf)


### Prerequisite
Notebook 02

------------------------------------

In [ ]:
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sbn
import scipy.stats as stats

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/akmand/datasets/master/openintro/evals.csv")
display(data)

## Course Evaluation Dataset

This data is part of the [OpenIntro resources](https://www.openintro.org/book/statdata/?data=evals).

"The data are gathered from end of semester student evaluations for a large sample of professors from the University of Texas at Austin. In addition, six students rate the professors' physical appearance. The result is a data frame where each row contains a different course and each column has information on either the course or the professor."

####Variables
- score: Average professor evaluation score: (1) very unsatisfactory - (5) excellent.
- rank: Rank of professor: teaching, tenure track, tenured.
- ethnicity: Ethnicity of professor: not minority, minority.
- gender: Gender of professor: female, male.
- language: Language of school where professor received education: English or non-English.
- age: Age of professor.
- cls_perc_eval: Percent of students in class who completed evaluation.
- cls_did_eval: Number of students in class who completed evaluation.
- cls_students: Total number of students in class.
- cls_level: Class level: lower, upper.
- cls_profs: Number of professors teaching sections in course in sample: single, multiple.
- cls_credits: Number of credits of class: one credit (lab, PE, etc.), multi credit.
- bty_f1lower: Beauty rating of professor from lower level female: (1) lowest - (10) highest.
- bty_f1upper: Beauty rating of professor from upper level female: (1) lowest - (10) highest.
- bty_f2upper: Beauty rating of professor from second level female: (1) lowest - (10) highest.
- bty_m1lower: Beauty rating of professor from lower level male: (1) lowest - (10) highest.
- bty_m1upper: Beauty rating of professor from upper level male: (1) lowest - (10) highest.
- bty_m2upper: Beauty rating of professor from second upper level male: (1) lowest - (10) highest.
- bty_avg: Average beauty rating of professor.
- pic_outfit: Outfit of professor in picture: not formal, formal.
- pic_color: Color of professor’s picture: color, black & white.

### Is there a difference in the mean course evaluation score between female and male instructors?

In [ ]:
sbn.boxplot(data = data,
            x="gender",
            y="score")
#sbn.pairplot(data); ## this will take awhile

# Simple Linear Regression

### Is there a statistically significant linear relationship between the beauty score and the course evaluation score?

In [ ]:
sbn.scatterplot(data=data,  x="bty_avg", y="score");

Fit a linear regression model and make a conclusion. Don't forget to check the conditions! (Slide 8, page 4 of Simple Linear Regression Notes)



# Multiple Regression with Binary Categorical Variable

### Is there a difference in the mean course evalution score between female and male instructors after accounting for the beauty score?

In [ ]:
sbn.scatterplot(data=data,  x="bty_avg", y="score", hue = 'gender');

In [ ]:
# Specify the model
model_formulation = smf.ols("score ~ bty_avg + gender", data = data)

# Fit the model
results = model_formulation.fit()

# View parameters
results.params

## Parallel Slopes Model Summary

Which coefficient should we do inference on to answer our question of interest?

In [ ]:
model_summary = results.summary()
print(model_summary)

model_anova = statsmodels.stats.anova.anova_lm(results)
print(model_anova)

### Parallel Slopes Predicted Values

In [ ]:
# Retrieve beta estimates from statsmodels
beta_hat = results.params

# split into groups
males = data[data.gender == "male"]
#males.info()

females = data[data.gender == "female"]
#females.info()

# Compute estimates
Y_hat_M = beta_hat[0] + beta_hat[1] + males.bty_avg*beta_hat[2]
Y_hat_F = beta_hat[0] + females.bty_avg*beta_hat[2]



In [ ]:
males['predVal'] = Y_hat_M
females['predVal'] = Y_hat_F



In [ ]:
# Show data points
sbn.scatterplot(data=data,  x="bty_avg", y="score", hue = 'gender');

# Graph the line of best fit
sbn.lineplot(data = males, x="bty_avg", y='predVal', color='blue')
sbn.lineplot(data = females, x="bty_avg", y='predVal', color='orange')

### Parallel Slopes Residuals

We still can't forget to check the residuals. What condition does the following code chunk assess? Do you believe the condition is met?

In [ ]:
residuals = results.resid
data['resid'] = residuals

plot = sbn.scatterplot(data=data,  x="bty_avg", y="resid", hue = 'gender');
plot.axhline(y=0, color = 'black')

What condition does the following code chunk assess? Do you believe the condition is met?

In [ ]:
results.resid.mean()

What condition does the following code chunk assess? Do you believe the condition is met?

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
sbn.distplot(residuals, fit=stats.norm, ax=ax1)
ax1.set_title('Distribution of Residuals')
ax1.set_xlabel('Residual')
ax1.set_ylabel('Probability')
_ = stats.probplot(residuals.ravel(), plot=ax2)

### Is the relationship between beauty score and course evaluation score the same for female and male instructors?

In [ ]:
# Specify the model
model_formulation = smf.ols("score ~ bty_avg * gender", data = data)

# Fit the model
results = model_formulation.fit()

# View parameters
results.params

### Interaction Model Summary

Which coefficient should we do inference on to answer our question of interest?

In [ ]:
model_summary = results.summary()
print(model_summary)

model_anova = statsmodels.stats.anova.anova_lm(results)
print(model_anova)

### Interaction Predicted Values

In [ ]:
# Retrieve beta estimates from statsmodels
beta_hat = results.params

# Compute estimates
Y_hat_M2 = beta_hat[0] + beta_hat[1] + males.bty_avg*(beta_hat[2] + beta_hat[3])
Y_hat_F2 = beta_hat[0] + females.bty_avg*beta_hat[2]

males['predVal2'] = Y_hat_M2
females['predVal2'] = Y_hat_F2

In [ ]:
# Show data points
sbn.scatterplot(data=data,  x="bty_avg", y="score", hue = 'gender');
# Graph the line of best fit
sbn.lineplot(data = males, x="bty_avg", y='predVal2', color='blue')
sbn.lineplot(data = females, x="bty_avg", y='predVal2', color='orange')

### Interaction Residuals

Are the conditions for inference met?

In [ ]:
residuals = results.resid
data['resid'] = residuals

plot = sbn.scatterplot(data=data,  x="bty_avg", y="resid", hue = 'gender');
plot.axhline(y=0, color = 'black')

results.resid.mean()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
sbn.distplot(residuals, fit=stats.norm, ax=ax1)
ax1.set_title('Distribution of Residuals')
ax1.set_xlabel('Residual')
ax1.set_ylabel('Probability')
_ = stats.probplot(residuals.ravel(), plot=ax2)

# Multiple Regression with Categorical Variable with more than two levels

In [ ]:
sbn.scatterplot(data=data,  x="bty_avg", y="score", hue = 'rank');

What statistical question can we ask using the variables displayed here?

In [ ]:
# Specify the model
model_formulation = smf.ols("score ~ bty_avg * rank", data = data)

# Fit the model
results = model_formulation.fit()

# View parameters
results.params

What do you conclude about your question of interest?

In [ ]:
model_summary = results.summary()
print(model_summary)

model_anova = statsmodels.stats.anova.anova_lm(results)
print(model_anova)

In [ ]:
# Retrieve beta estimates from statsmodels
beta_hat = results.params

# split into groups
#tt = data[data.rank == 'tenure track']
tt = data.loc[data['rank'] == 'tenure track']
tn = data.loc[data['rank'] == "tenured"]
te = data.loc[data['rank'] == "teaching"]

# Compute estimates
Y_hat_TT = beta_hat[0] + beta_hat[1] + tt.bty_avg*(beta_hat[3] + beta_hat[4])
Y_hat_TN = beta_hat[0] + beta_hat[2] + tn.bty_avg*(beta_hat[3] + beta_hat[5])
Y_hat_TE = beta_hat[0] + te.bty_avg*beta_hat[3]

tt['predVal'] = Y_hat_TT
tn['predVal'] = Y_hat_TN
te['predVal'] = Y_hat_TE

In [ ]:
# Show data points
sbn.scatterplot(data=data,  x="bty_avg", y="score", hue = 'rank');
# Graph the line of best fit
sbn.lineplot(data = tt, x="bty_avg", y='predVal', color='blue')
sbn.lineplot(data = tn, x="bty_avg", y='predVal', color='orange')
sbn.lineplot(data = te, x="bty_avg", y='predVal', color='green')

In [ ]:
residuals = results.resid
data['resid'] = residuals

plot = sbn.scatterplot(data=data,  x="bty_avg", y="resid", hue = 'rank');
plot.axhline(y=0, color = 'black')

results.resid.mean()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
sbn.distplot(residuals, fit=stats.norm, ax=ax1)
ax1.set_title('Distribution of Residuals')
ax1.set_xlabel('Residual')
ax1.set_ylabel('Probability')
_ = stats.probplot(residuals.ravel(), plot=ax2)

# Multiple Regression with Multiple Quantitative Variables

### Is there a statistically significant linear relationship between the beauty score and the course evaluation score after accounting for instructor age?

In [ ]:
sbn.scatterplot(data=data,  x="bty_avg", y="score");

In [ ]:
sbn.scatterplot(data=data,  x="age", y="score");

In [ ]:
sbn.scatterplot(data=data,  x="age", y="bty_avg");

## Check your understanding

 Consider all of the models presented so far. What are the different elements of statistical output that can be used as evidence that the model is ``good"? Which do you find most convincing? Why?

### Multicollinearity

Do we need to worry about age and bty_avg being redundant (i.e. explain similar variability in score)?

We can compute a variance inflation factor:

$$VIF = \frac{1}{1-R^2}$$

Where R^2 is the R^2 we have seen before in a regression model of one covariate against another.

A rule of thumb is an acceptable VIF is < 5.

In [ ]:
model_formulation = smf.ols("bty_avg ~ age", data = data)
model_summary = results.summary()
print(model_summary)

vif = 1/(1-0.0.059)
vif

In [ ]:
# Specify the model
model_formulation = smf.ols("score ~ bty_avg + age", data = data)

# Fit the model
results = model_formulation.fit()

# View parameters
results.params

In [ ]:
model_summary = results.summary()
print(model_summary)

model_anova = statsmodels.stats.anova.anova_lm(results)
print(model_anova)

In [ ]:
residuals = results.resid

data['resid'] = residuals

predVal = results.params[0] + results.params[1]*data.bty_avg + results.params[2]*data.age

data['predVal'] = predVal

plot = sbn.scatterplot(data=data,  x="predVal", y="resid");
plot.axhline(y=0, color = 'black')

results.resid.mean()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
sbn.distplot(residuals, fit=stats.norm, ax=ax1)
ax1.set_title('Distribution of Residuals')
ax1.set_xlabel('Residual')
ax1.set_ylabel('Probability')
_ = stats.probplot(residuals.ravel(), plot=ax2)